In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your_client_id_here",
).login()

In [ ]:
from evo.objects.helpers import EvoContext

evo_context = EvoContext.from_environment(
    connector=manager.get_connector(),
    environment=manager.get_environment(),
    cache=manager.cache,
)

In [ ]:
import uuid

import pandas as pd

from evo.objects.helpers.typed.pointset import PointSet, PointSetData

point_set_data = PointSetData(
    name=f"My Point Set Data - {uuid.uuid4()}",
    data=pd.DataFrame(
        {
            "x": [1.0, 2.0, 3.0],
            "y": [4.0, 5.0, 6.0],
            "z": [7.0, 8.0, 9.0],
            "a": [10.0, 11.0, 12.0],
            "b": [False, True, False],
            "c": ["point1", "point2", "point3"],
        }
    ),
    coordinate_reference_system="EPSG:4326",
)
point_set = await PointSet.create(
    evo_context=evo_context,
    data=point_set_data,
)

df = await point_set.as_dataframe()
df

In [ ]:
for attribute in point_set.attributes:
    if attribute.name == "a":
        print(attribute.key)
        print(await attribute.as_dataframe())
        break

In [ ]:
point_set.description = "This is my point set."
df.loc[0, "a"] = 99.9
df["d"] = [100, 200, 300]
await point_set.update_attributes(df)
await point_set.update()

In [ ]:
import json

from evo_schemas.elements.serialiser import GSONEncoder

from evo.objects.helpers.geoscience_object import BaseObject

o = await BaseObject.from_reference(
    evo_context=evo_context,
    reference=point_set.metadata.url,
)
print(type(o))

print(o.metadata)
print(json.dumps(o.as_dict(), indent=2, cls=GSONEncoder))

df = await o.as_dataframe()
df

In [ ]:
point_set_data = PointSetData(
    name=f"My Point Set Data - {uuid.uuid4()}",
    data=pd.DataFrame(
        {
            "x": [11.0, 2.0, 3.0],
            "y": [4.0, 5.0, 6.0],
            "z": [7.0, 8.0, 9.0],
            "a": [10.0, 11.0, 12.0],
            "b": [False, True, False],
            "c": ["point1", "point2", "point3"],
        }
    ),
    coordinate_reference_system="EPSG:4326",
)
new_point_set = await PointSet.replace(
    evo_context=evo_context,
    reference=point_set.metadata.url,
    data=point_set_data,
)
df = await new_point_set.as_dataframe()
df

In [ ]:
from evo.objects.exceptions import ObjectModifiedError

# Should raise ObjectModifiedError because 'o' is stale
try:
    await o.update()
except ObjectModifiedError as e:
    print(f"ObjectModifiedError: {e}")
else:
    print("No ObjectModifiedError raised")

In [ ]:
from evo.objects.helpers.typed.grid import Regular3DGrid, Regular3DGridData
from evo.objects.helpers.types import Point3, Size3d, Size3i

grid_data = Regular3DGridData(
    name=f"My Regular 3D Grid Data - {uuid.uuid4()}",
    origin=Point3(0.0, 0.0, 0.0),
    size=Size3i(3, 3, 3),
    cell_size=Size3d(1.0, 1.0, 1.0),
    cell_data=pd.DataFrame(
        {
            "a": [10.0, 11.0, 12.0],
            "b": [False, True, False],
            "c": ["point1", "point2", "point3"],
        }
    ),
    coordinate_reference_system="EPSG:4326",
)
grid = await Regular3DGrid.create(
    evo_context=evo_context,
    data=grid_data,
)
print([attr.name for attr in grid.cells.attributes])
grid_df = await grid.cells.as_dataframe()
grid_df